In [37]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

import seaborn
from matplotlib import pyplot

from sklearn.linear_model import LogisticRegression

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [54]:
chess_games = pd.read_csv('chess_games.csv')

In [55]:
display(chess_games)
#chess_games['Event'] = chess_games['Event'].astype(str)
chess_games['UTCDate'] = pd.to_datetime(chess_games['UTCDate'])
chess_games['UTCTime'] = pd.to_datetime(chess_games['UTCTime'])
#chess_games['ECO'] = chess_games['ECO'].astype('|S')
print(chess_games.dtypes)

#change for github demo

,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256179,Blitz,GustavScar,WEBRIFF,1-0,2016.07.31,21:59:56,1248,1303,13.0,-17.0,B54,Sicilian Defense,180+3,Normal,1. e4 c5 2. Nf3 d6 3. d4 Qa5+ 4. Bd2 Qc7 5. Bc...
6256180,Classical,apgaro,Camposf,1-0,2016.07.31,21:59:54,1328,1292,10.0,-11.0,C40,King's Knight Opening,1800+0,Normal,1. e4 e5 2. Nf3 h6 3. Nc3 d6 4. Bc4 Qf6 5. Nd5...
6256181,Bullet,matematico1234567890,cecile_creteil,0-1,2016.07.31,21:59:53,1660,1658,-11.0,11.0,B21,Sicilian Defense: Smith-Morra Gambit,120+0,Normal,1. e4 { [%eval 0.22] } 1... c5 { [%eval 0.35] ...
6256182,Bullet,MagnusMinor,shegap,0-1,2016.07.31,21:59:57,1726,1776,-8.0,9.0,A09,Reti Opening: Reti Accepted,60+0,Normal,1. Nf3 d5 2. c4 dxc4 3. Qa4+ c6 4. Qxc4 Nf6 5....


Event                      object
White                      object
Black                      object
Result                     object
UTCDate            datetime64[ns]
UTCTime            datetime64[ns]
WhiteElo                    int64
BlackElo                    int64
WhiteRatingDiff           float64
BlackRatingDiff           float64
ECO                        object
Opening                    object
TimeControl                object
Termination                object
AN                         object
dtype: object


In [61]:

#%sql SELECT * FROM chess_games WHERE Result = '*'
%sql SELECT DISTINCT Termination FROM chess_games
#%sql SELECT * FROM chess_games WHERE Termination = 'Rules infraction'
#filter out Termination = Abandoned, Rules infraction, filtering out TimeControl = '-'

#filter out all NaN values

#separating TimeControl into 2 columns (base & additional)

,Termination
0,Normal
1,Time forfeit
2,Abandoned
3,Rules infraction
4,Unterminated


In [57]:
#Summary Statistics and Descriptive Statistics
#%sql top_ten_openings << SELECT Opening, COUNT(White) AS Games_Won FROM chess_games WHERE Result = '1-0' GROUP BY White, Opening ORDER BY Games_Won DESC LIMIT 10



,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Bullet,TotalPatzer,PeterV,*,2016-07-02,2022-10-17 19:53:51,2277,2101,NaN,NaN,B12,"Caro-Kann Defense: Maroczy Variation, Maroczy ...",60+0,Abandoned,1. e4 c6 2. d4 d5 3. f3 dxe4 4. fxe4 e5 5. Nf3...
1,Classical,VW_STO,joze12,*,2016-07-04,2022-10-17 06:31:35,1770,1579,NaN,NaN,C00,French Defense #2,900+10,Abandoned,1. e4 e6 2. a3 h6 3. Bc4 Qh4 4. d3 a6 5. Nc3 b...
2,Classical,Mochialf,sohrab-sad,*,2016-07-04,2022-10-17 06:48:46,1858,1933,NaN,NaN,A16,"English Opening: Anglo-Indian Defense, Queen's...",600+0,Abandoned,1. c4 Nf6 2. Nc3 g6 3. Nf3 Bg7 4. g3 O-O 5. Bg...
3,Classical,SME,SEM7891,*,2016-07-04,2022-10-17 06:50:31,1330,1387,NaN,NaN,A00,Van't Kruijs Opening,600+0,Abandoned,1. e3 d5 2. c4 Nf6 3. Qb3 Bf5 4. Nc3 b6 5. Nxd...
4,Classical,VENEZUELALIBRE,romka0880,*,2016-07-04,2022-10-17 06:50:34,1574,1660,NaN,NaN,C20,King's Pawn Game: Napoleon Attack,600+0,Abandoned,1. e4 e5 2. Qf3 Nf6 3. d3 Nc6 4. c3 b6 5. Be2 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,Bullet,Serg_tourn,susguan,*,2016-07-31,2022-10-17 07:26:01,2089,1689,NaN,NaN,E87,"King's Indian Defense: Saemisch Variation, Clo...",60+0,Abandoned,1. c4 g6 2. Nc3 Bg7 3. d4 e5 4. d5 d6 5. e4 Nf...
1339,Bullet,irinas,wey,*,2016-07-31,2022-10-17 07:26:06,1949,1945,NaN,NaN,B20,Sicilian Defense,60+0,Abandoned,1. e4 c5 2. d3 Nc6 3. Nc3 g6 4. Be3 Bg7 5. Qd2...
1340,Bullet,lufo,indian_heat,*,2016-07-31,2022-10-17 07:27:40,2079,1938,NaN,NaN,A04,Zukertort Opening: Queen's Gambit Invitation,30+0,Abandoned,1. Nf3 e6 2. d4 d5 3. c4 Be7 4. e3 c6 5. Nc3 N...
1341,Correspondence,elvan,AC8,*,2016-07-31,2022-10-17 13:45:07,1677,1508,NaN,NaN,B15,Caro-Kann Defense,-,Abandoned,1. e4 c6 2. d4 d5 3. Nc3 dxe4 *
